# Import

In [21]:
import csv
import os
import sys

import numpy as np
import pandas as pd
from nltk import word_tokenize, download
from stop_words import get_stop_words
from nltk.corpus import stopwords
from tqdm import tqdm

## Add configuration file

In [7]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [8]:
from ALL import config 
from util import *

## Set condition

In [9]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [10]:
s3 = S3Manager()

# Read data

In [15]:
df_path = s3.download("DataShaping/AgNews/master.csv")

In [16]:
df = pd.read_csv(df_path, index_col=0)

In [18]:
labels_path = s3.download("DataShaping/AgNews/class.csv")

In [19]:
with open(labels_path, mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

# Word tokinize

In [25]:
download('punkt')
download("stopwords")

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [28]:
df["words"] = df.title.progress_apply(word_tokenize)

100%|██████████| 120000/120000 [00:12<00:00, 9315.36it/s]


In [29]:
stop_words_add = ["would", "could", "should"]
stop_char = ["==", "--", "\'s", "''", "n't", "``","..", "...", "....", "'m", "'ve","'re", "'d", "'ll", "", "-+", "+-", "_/", "||", "__", "/|", "//"]
stop_words = set(stopwords.words("english") + get_stop_words("english") + stop_words_add + stop_char)

In [30]:
#     一文字以下の単語とstop_word, stop_charを削除
df["words_nonstop"] = df.words.progress_apply(
    lambda words: [
        word for word in words if word.lower() not in stop_words if len(word)> 1
    ]
)

100%|██████████| 120000/120000 [00:00<00:00, 286070.20it/s]


In [31]:
df.words = df.words.progress_apply(lambda words: " ".join(words))
df.words_nonstop = df.words_nonstop.progress_apply(
    lambda words: " ".join(words)
)

100%|██████████| 120000/120000 [00:00<00:00, 609822.61it/s]


In [33]:
df = df.rename(mapper={"text": "_text", "title": "text"}, axis=1)
# ag_news_df = ag_news_df.drop(ag_news_df[ag_news_df.words_nonstop == ""].index)

# output

## make file

In [35]:
with open(make_filepath(f"../../temporary/Preprocessing/AgNewsTitle/class.csv"), "w") as f:
    writer = csv.writer(f)
    writer.writerow(class_labels)
df.to_csv(make_filepath(f"../../temporary/DataShaping/AgNewsTitle/master.csv"))

## upload file

In [36]:
s3.upload("../../temporary/Preprocessing/AgNewsTitle", "DataShaping/AgNewsTitle/")

'../../temporary/Preprocessing/AgNewsTitle'

In [37]:
s3.delete_local_all()

../../temporary/Preprocessing/AgNewsTitle
/home/jovyan//temporary/DataShaping/AgNews/master.csv
/home/jovyan//temporary/DataShaping/AgNews/class.csv


In [38]:
shutil.rmtree("../../nltk_data/")